In [206]:
import pandas as pd

##### 0. base 작업
Respiratory 진단 받은 환자의 SUBJECT_ID, HADM_ID, TLOS, AGE, DOA(Dead or Alive) column 생성 및 추출(총 7,631명)

In [227]:
# respiratory 환자 사망률
print("호흡기 질환 환자의 사망률")
print("-"*59)
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/D_ICD_DIAGNOSES.csv")
word = ["respiratory", "Respiratory"]
disease = '|'.join(word)
df1 = df[df['LONG_TITLE'].str.contains(disease)]
disease_list = df1["ICD9_CODE"].to_list()
print("Respiratory 관련 ICD9 code 수: ", len(disease_list))
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/DIAGNOSES_ICD.csv")

def add(group):
    return ' '.join(group['ICD9_CODE'].fillna(""))

df1 = df.groupby(['SUBJECT_ID', 'HADM_ID']).apply(add).reset_index(name="ICD9_CODE")

mask = df1['ICD9_CODE'].str.contains('|'.join(disease_list), na=False)
df2 = df1[mask]

# 동일한 환자가 여러 번 입원한 경우 최초 입원 기록만 남기기
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/ADMISSIONS.csv")
df3 = df[["HADM_ID", "ADMITTIME", "DISCHTIME", "DEATHTIME"]]
df4 = pd.merge(df2, df3, on="HADM_ID")
df4['ADMITTIME'] = pd.to_datetime(df4['ADMITTIME'])
df4['DISCHTIME'] = pd.to_datetime(df4['DISCHTIME'])
df4['DEATHTIME'] = pd.to_datetime(df4['DEATHTIME'])
df5 = df4.sort_values(by=['SUBJECT_ID', 'ADMITTIME'], ascending=[True, True])
df5 = df5.drop_duplicates(subset=['SUBJECT_ID'], keep='first')
df5.sort_values(by=["SUBJECT_ID"], ascending=True, inplace=True)
print("Respiratory 환자 수(최초 입원): ", len(df5))

# LOS 추가 후 24H 이상 머문 환자 list만 저장
df5["TLOS"] = df5["DISCHTIME"] - df5["ADMITTIME"]
over = df5["TLOS"] >= "1days"
df6 = df5[over].sort_values(by=["TLOS"], ascending=True)
subject_list = df6["SUBJECT_ID"].to_list()
print("TLOS가 24H 이상인 환자 수: ", len(subject_list))

# 첫 입원 시점 연령을 구하기 위해 PATIENTS Table에서 DOB 칼럼 가져오기
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/PATIENTS.csv")
df = df[["SUBJECT_ID", "DOB"]]
df['DOB'] = pd.to_datetime(df['DOB']).dt.date
df7 = pd.merge(df6, df, on="SUBJECT_ID")

# 첫 입원 시점 연령 칼럼 생성
df7["AGE"] = ((df7["ADMITTIME"].dt.date - df7["DOB"])//365).dt.days

# 18세 이상 환자 선별
over = df7["AGE"] >= 18
df8 = df7[over].sort_values(by=["AGE"], ascending=True)
subject_list = df8["SUBJECT_ID"].to_list()

# 89세 이상 환자 제거
over = df8["AGE"] < 89
df9 = df8[over].sort_values(by=["AGE"], ascending=True)
final_list = df9["SUBJECT_ID"].to_list()
print("18세 이상 89세 미만 환자 수: ", len(final_list))

# DOA(Death or Alive): 사망했다면 사망까지 걸린 시간, 생존했다면 NaT 출력
for i in range(len(df9)):
    if df9["DEATHTIME"].iloc[i] != "NaT":
        df9["DOA"] = (df9["DEATHTIME"]-df9["ADMITTIME"])
    else:
        df9["DOA"] = "NaT"

df10 = df9[["SUBJECT_ID", "HADM_ID", "TLOS", "AGE", "DOA"]]
print("-"*59)
print("호흡기 질환 환자의 가공된 Admission table dataframe")
df10

호흡기 질환 환자의 사망률
-----------------------------------------------------------
Respiratory 관련 ICD9 code 수:  92
Respiratory 환자 수(최초 입원):  10329
TLOS가 24H 이상인 환자 수:  9932
18세 이상 89세 미만 환자 수:  7631
-----------------------------------------------------------
호흡기 질환 환자의 가공된 Admission table dataframe


,SUBJECT_ID,HADM_ID,TLOS,AGE,DOA
6392,20936,157548,16 days 06:15:00,18,NaT
582,25600,122994,2 days 12:49:00,18,NaT
1012,79166,177249,3 days 11:02:00,18,NaT
3214,75775,101668,7 days 13:07:00,18,NaT
5057,26861,115381,11 days 16:01:00,18,NaT
...,...,...,...,...,...
6091,48806,116558,14 days 23:36:00,88,NaT
3008,19067,192309,7 days 01:22:00,88,NaT
2138,40370,199999,5 days 12:33:00,88,NaT
6524,47335,161669,16 days 20:28:00,88,16 days 20:28:00


##### 1. 인구통계학 데이터 추출 1
-> SUBJECT_ID, HADM_ID, AGE, GENDER, ETHNICITY, TLOS(병원 전체), LOS(ICU 전체), DOA(사망일까지 걸린 시간), HOSPITAL_EXPIRE_FLAG(병원 내 사망 기록) column 추출(총 7,608명)
1. ADMISSIONS(가공)
2. ADMISSIONS(원본)
3. PATIENTS
4. DIAGNOSES_ICD

In [261]:
df1 = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/ADMISSIONS.csv")
df2 = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/PATIENTS.csv")
df3= pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/ICUSTAYS.csv")

In [279]:
# Respiratory 진단 받은 환자의 AMISSIONS, PATIENTS, ICUSTAYS 합치기
# ICUSTAYS merge 과정에서
print("ADMISSIONS(가공) table 환자 수: ", len(df10))
df6 = pd.merge(df10, df1, on=['SUBJECT_ID', 'HADM_ID']).sort_values(by='SUBJECT_ID')
print("ADMISSIONS(원본) table 추가: ", len(df6))
df7 = pd.merge(df6, df2, on=['SUBJECT_ID'])
print("PATIENTS table 추가: ", len(df7))
df8 = df3[["SUBJECT_ID", "HADM_ID", "LOS"]]
df8 = df8.groupby(["SUBJECT_ID","HADM_ID"])['LOS'].sum().reset_index()
df9 = pd.merge(df7, df8, on=['SUBJECT_ID', "HADM_ID"]).sort_values(by='SUBJECT_ID')
print("ICUSTAYS table 추가: ", len(df9))
print("-"*117)
df = df9.drop(["ROW_ID_x", "ROW_ID_y", "ADMITTIME", "DISCHTIME", "DEATHTIME", "DOB", "DOD", "DOD_HOSP", "DOD_SSN", "EXPIRE_FLAG", "HAS_CHARTEVENTS_DATA", "EDREGTIME", "EDOUTTIME", "RELIGION", "LANGUAGE", "MARITAL_STATUS", "ADMISSION_LOCATION", "DISCHARGE_LOCATION", "INSURANCE", "ADMISSION_TYPE"], axis=1)
df = df.reindex(columns=["SUBJECT_ID","HADM_ID", "AGE", "GENDER", "ETHNICITY", "TLOS", "LOS", "DOA","HOSPITAL_EXPIRE_FLAG"])
print("활력징후 데이터 column: ", len(df.columns))
print("활력징후 데이터 column name: ", df.columns.values)
print("-"*117)
print("호흡기 질환 환자의 인구통계학 데이터 table")
df

ADMISSIONS(가공) table 환자 수:  7631
ADMISSIONS(원본) table 추가:  7631
PATIENTS table 추가:  7631
ICUSTAYS table 추가:  7608
---------------------------------------------------------------------------------------------------------------------
활력징후 데이터 column:  9
활력징후 데이터 column name:  ['SUBJECT_ID' 'HADM_ID' 'AGE' 'GENDER' 'ETHNICITY' 'TLOS' 'LOS' 'DOA'
 'HOSPITAL_EXPIRE_FLAG']
---------------------------------------------------------------------------------------------------------------------
호흡기 질환 환자의 인구통계학 데이터 table


,SUBJECT_ID,HADM_ID,AGE,GENDER,ETHNICITY,TLOS,LOS,DOA,HOSPITAL_EXPIRE_FLAG
0,36,165660,72,M,WHITE,10 days 01:46:00,4.3483,NaT,0
1,91,121205,81,F,WHITE,17 days 15:08:00,3.9715,17 days 15:08:00,1
2,94,140037,75,M,ASIAN,22 days 22:53:00,22.8977,NaT,0
3,101,175533,82,M,ASIAN,15 days 18:41:00,9.8919,15 days 18:41:00,1
4,111,192123,66,F,WHITE,11 days 04:50:00,10.5708,NaT,0
...,...,...,...,...,...,...,...,...,...
7603,99897,181057,54,M,BLACK/HAITIAN,5 days 00:02:00,0.8342,NaT,0
7604,99899,188409,87,M,BLACK/AFRICAN AMERICAN,9 days 01:34:00,9.1439,9 days 01:34:00,1
7605,99912,189380,84,M,WHITE,13 days 21:35:00,12.0867,NaT,0
7606,99939,159023,26,M,HISPANIC OR LATINO,11 days 10:59:00,2.8780,NaT,0


In [290]:
# ICUSTAYS table merge 과정에서 환자 수 줄어드는 이유 파악하기 위한 코드
list1=df6["SUBJECT_ID"].drop_duplicates().tolist() # ADMISSIONS(원본) table 추가
list2=df7["SUBJECT_ID"].drop_duplicates().tolist() # PATIENTS table 추가
list3=df8["SUBJECT_ID"].drop_duplicates().tolist() # ICUSTAYS unique value
list4=df9["SUBJECT_ID"].drop_duplicates().tolist() # ICUSTAYS table 추가

print(len(list1))
print(len(list2))
print(len(list3))
print(len(list4))
print("-"*10)


common = []
for item in list1:
    if item in list2:
        common.append(item)

print(len(common))

common1 = []
for item in list2:
    if item in list3:
        common1.append(item)

print(len(common1))

common2 = []
for item in list3:
    if item in list4:
        common2.append(item)

print(len(common2))

# result = [x for x in common if x not in common1]
# result_df = df8[df8['SUBJECT_ID'].isin(result)]
# result_df

7631
7631
46476
7608
----------
7631
7629
7608


##### 2. 인구통계학 데이터 추출 2
-> CHARTEVENT table에서 체중, 키 및 체질량 지수(BMI)column 추가

##### 3. 호흡기 질환 환자의 사망률 확인

In [292]:
# 특정 기간내 사망률 확인
# 30일 이내 사망
less = df["DOA"] <= "30days"
df11 = df[less].sort_values(by=["DOA"], ascending=True)
subject_list = df11["SUBJECT_ID"].to_list()
print("30일 이내 사망자 수: ", len(subject_list))
print("30일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
print("-----------------------------------------")

# 60일 이내 사망
less = df["DOA"] <= "60days"
df12 = df[less].sort_values(by=["DOA"], ascending=True)
subject_list = df12["SUBJECT_ID"].to_list()
print("60일 이내 사망자 수: ", len(subject_list))
print("60일 이내 사망자 수 비율: ", len(subject_list)/len(final_list)*100)
print("-----------------------------------------")

30일 이내 사망자 수:  1741
30일 이내 사망자 수 비율:  22.814834228803566
-----------------------------------------
60일 이내 사망자 수:  1904
60일 이내 사망자 수 비율:  24.950858340977593
-----------------------------------------
